In [ ]:
import numpy as np 
import pandas as pd 
import fastai
from fastai.tabular.all import *

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
sub = pd.read_csv("../input/tabular-playground-series-jun-2021/sample_submission.csv")

features = list(set(train.columns).difference({'target','id'}))

In [ ]:
n=40
pred = sub[sub.columns[1:]]*0
log = pd.DataFrame(columns=['Loss Train','Loss Validation','Accuracy'])

for i in range(n):
    splits = RandomSplitter(valid_pct=0.05)(range_of(train))
    
    to = TabularPandas(
        train,
        y_names="target",
        y_block = CategoryBlock,
        cat_names = features,
        procs = [Categorify],
        splits=splits)
    
    loaders = to.dataloaders(bs=256)
    learn = tabular_learner(loaders, metrics=accuracy)
    learn.fit_one_cycle(3,0.01,wd=0.1)
    loss = learn.recorder.log[1:4]
    log.loc[len(log)] = loss
    dl = learn.dls.test_dl(test[features])
    pred+= learn.get_preds(dl=dl)[0].numpy()/n

log.describe()

In [ ]:
log.describe()

In [ ]:
sub[sub.columns[1:]] = pred
sub.to_csv("submission.csv", index=False)